In [1]:
import tensorflow as tf

In [3]:
import os
import pandas as pd
import numpy as np

In [4]:
for dirName,DontKnow,filename in os.walk(os.getcwd()):
    print(dirName)

/Users/pravin10717/Jupyter Project
/Users/pravin10717/Jupyter Project/JN2Data
/Users/pravin10717/Jupyter Project/JN2Data/five_images
/Users/pravin10717/Jupyter Project/JN1Data
/Users/pravin10717/Jupyter Project/JN1Data/test1
/Users/pravin10717/Jupyter Project/JN1Data/train
/Users/pravin10717/Jupyter Project/JN1Data/train/dogs
/Users/pravin10717/Jupyter Project/JN1Data/train/cats
/Users/pravin10717/Jupyter Project/JN1Data/validation
/Users/pravin10717/Jupyter Project/JN1Data/validation/dogs
/Users/pravin10717/Jupyter Project/JN1Data/validation/cats
/Users/pravin10717/Jupyter Project/.ipynb_checkpoints


In [5]:
test_directory="JN1Data/test1/"
train_directory="JN1Data/train/"
validation_directory="JN1Data/validation/"

In [6]:
import shutil

In [14]:
os.mkdir(validation_directory)

In [15]:
for dir_name in ["cats","dogs"]:
    os.mkdir(train_directory+dir_name)
    os.mkdir(validation_directory+dir_name)

In [18]:
file_list=os.listdir(train_directory)
for file_name in file_list:
    if(file_name.startswith("cat")):
        shutil.move(train_directory+file_name,train_directory+"cats")
    if(file_name.startswith("dog")):
        shutil.move(train_directory+file_name,train_directory+"dogs")

In [8]:
ls JN1Data/train/cats | wc -l

    6500


In [9]:
ls JN1Data/train/dogs | wc -l

    6500


In [ ]:
for i in range(0,6000):
    shutil.move(train_directory+"cats/cat."+str(i)+".jpg",validation_directory+"cats")
    shutil.move(train_directory+"cats/cat."+str(i)+".jpg",validation_directory+"cats")

In [10]:
tf.__version__

'2.2.0'

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense,BatchNormalization
from tensorflow.keras.models import Sequential

In [12]:
train_generator=ImageDataGenerator(rescale=1./255)
validation_generator=ImageDataGenerator(rescale=1./255)

In [13]:
train_generator_data=train_generator.flow_from_directory(train_directory,
                                                        target_size=(150,150),
                                                        batch_size=15,
                                                        class_mode="categorical")
validation_generator_data=validation_generator.flow_from_directory(validation_directory,
                                                                  target_size=(150,150),
                                                                  batch_size=15,
                                                                  class_mode="categorical")

Found 13000 images belonging to 2 classes.
Found 12000 images belonging to 2 classes.


In [17]:
model=Sequential([Conv2D(filters=32,kernel_size=(4,4),input_shape=(150,150,3),activation="relu"),
                 BatchNormalization(),
                 MaxPooling2D(pool_size=(2,2)),
                 Dropout(0.25),
                 Conv2D(filters=64,kernel_size=(4,4),activation="relu"),
                 BatchNormalization(),
                 MaxPooling2D(pool_size=(2,2)),
                 Dropout(0.25),
                 Conv2D(filters=128,kernel_size=(4,4),activation="relu"),
                 BatchNormalization(),
                 MaxPooling2D(pool_size=(2,2)),
                 Dropout(0.5),
                 Flatten(),
                 Dense(units=512,activation="relu"),
                 BatchNormalization(),
                 Dropout(0.5),
                 Dense(units=2,activation="softmax")]
                )

In [14]:
model=Sequential([Conv2D(filters=16,kernel_size=(3,3),input_shape=(150,150,3),activation="relu"),
                 MaxPooling2D(2,2),
                 Conv2D(filters=32,kernel_size=(3,3),activation="relu"),
                 MaxPooling2D(2,2),
                 Conv2D(filters=64,kernel_size=(3,3),activation="relu"),
                 MaxPooling2D(2,2),
                 Flatten(),
                 Dense(units=512,activation="relu"),
                 Dense(units=2,activation="softmax")]
                )

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 147, 147, 32)      1568      
_________________________________________________________________
batch_normalization_16 (Batc (None, 147, 147, 32)      128       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 70, 70, 64)        32832     
_________________________________________________________________
batch_normalization_17 (Batc (None, 70, 70, 64)        256       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 35, 35, 64)        0

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [16]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["acc"])

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

In [18]:
earlyStopping=EarlyStopping(patience=5)

In [ ]:
history=model.fit(train_generator_data,epochs=10,validation_data=validation_generator_data,callbacks=[earlyStopping])

Epoch 1/10
867/867 [==============================] - 876s 1s/step - loss: 0.7635 - acc: 0.6237 - val_loss: 0.6267 - val_acc: 0.6457
Epoch 2/10
626/867 [====================>.........] - ETA: 4:05 - loss: 0.5963 - acc: 0.6885

In [19]:
history=model.fit(train_generator_data,epochs=10,validation_data=validation_generator_data,callbacks=[earlyStopping])

Epoch 1/10
867/867 [==============================] - 283s 326ms/step - loss: 0.6705 - acc: 0.6102 - val_loss: 0.6067 - val_acc: 0.7011
Epoch 2/10
644/867 [=====================>........] - ETA: 53s - loss: 0.5300 - acc: 0.7385

KeyboardInterrupt: 